In [ ]:
#mkdir /content/tmp
#%cp -r -f -v /content/drive/MyDrive/SUMONBDT /content/tmp
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/SUMONBDT
#%cd /home/liuli/github/SUMONBDT
#!nvidia-smi
#用于测试oneHot
#############################################################也是第一步，读取数据
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())



enc = OneHotEncoder()
#[2,3,5,9]
x1 = [0,0,0,0]
x2 = [0,0,0,1]

x3 = [1,1,1,2]
x4 = [1,1,1,3]
x5 = [1,1,2,4]
x6 = [1,1,2,5]
x7 = [1,2,3,6]
x8 = [1,2,3,7]
x9 = [1,2,4,8]
X = [x1, x2, x3,x4,x5,x6,x7,x8,x9]
enc.fit(X)
#print(enc.transform(X).toarray())


########################读写CSV,并转为oneHot
file1 = "./trainData/dataAllSim10000.csv"
print("reading data")
xyDataTmp = pd.read_csv(file1)
#print(xyDataTmp.info())
xyData = np.array(xyDataTmp)

x = xyData[:,0:22]
y = xyData[:,22:26]

y = enc.transform(y).toarray()

print("x.shape:",x.shape,"yOneHot.shape:",y.shape)



del xyDataTmp #节省内存
del xyData #节省内存



In [ ]:

################################################################第二步，训练
#1. 核心为keras220不是pytorch
#2. 基于hmcnf
import model_hmcnf
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model

#os.environ["CUDA_VISIBLE_DEVICES"]="0"

#hierarchy = [18, 80, 178, 142, 77, 4]
hierarchy = [2,3,5,9]
features_size = x.shape[1]
label_size = y.shape[1]
beta = 0.2
dropout_rate=0.1
relu_size=384



def local_model(num_labels, dropout_rate, relu_size):
    model = tf.keras.Sequential()
    model.add(layers.Dense(relu_size, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_labels, activation='sigmoid'))
    return model


def global_model(dropout_rate, relu_size):
    model = tf.keras.Sequential()
    model.add(layers.Dense(relu_size, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    return model


def sigmoid_model(label_size):
    model = tf.keras.Sequential()
    model.add(layers.Dense(label_size, activation='sigmoid',name="global"))
    return model

features = layers.Input(shape=(features_size,))
global_models = []
local_models = []


for i in range(len(hierarchy)):
    if i == 0:
        global_models.append(global_model(dropout_rate, relu_size)(features))
    else:
        global_models.append(global_model(dropout_rate, relu_size)(layers.concatenate([global_models[i-1], features])))

p_glob = sigmoid_model(label_size)(global_models[-1])


#显示只有全局模型的情况
#modelTmp1 = tf.keras.Model(inputs=[features], outputs=[p_glob])
#modelTmp1.summary()#
#plot_model(modelTmp1, to_file='Flatten1.png', show_shapes=True)


for i in range(len(hierarchy)):
    local_models.append(local_model(hierarchy[i], dropout_rate, relu_size)(global_models[i]))
    
#显示只有局部局模型的情况(部分全局)
p_loc = layers.concatenate(local_models)
#modelTmp2 = tf.keras.Model(inputs=[features], outputs=[p_loc])
#modelTmp2.summary()#
#plot_model(modelTmp2, to_file='Flatten2.png', show_shapes=True)
p_glob1 = layers.Lambda(lambda x: x*beta,name="global")(p_glob)
p_loc1 = layers.Lambda(lambda x: x*(1-beta),name="local")(p_loc)

labels = layers.add([p_glob1, p_loc1])

model = tf.keras.Model(inputs=[features], outputs=[labels])




plot_model(model, to_file='FlattenAll.png', show_shapes=True)


model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),loss='binary_crossentropy',metrics=['mae'])
model.fit([x],[y],epochs=1000, batch_size=25600*1)
model.save("hmcnf10000.h5")

In [118]:
##################################################################第三步，验证
#%cd /content/drive/MyDrive/SUMONBDT
import model_hmcnf
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model
import numpy as np
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#######################0.准备onehot
enc = OneHotEncoder()
#[2,3,5,9]
x1 = [0,0,0,0]
x2 = [0,0,0,1]

x3 = [1,1,1,2]
x4 = [1,1,1,3]
x5 = [1,1,2,4]
x6 = [1,1,2,5]
x7 = [1,2,3,6]
x8 = [1,2,3,7]
x9 = [1,2,4,8]
X = [x1, x2, x3,x4,x5,x6,x7,x8,x9]
enc.fit(X)

#######################2.准备数据
        
file1 = "./trainData/dataAllSim10000.csv"
file1 = "./trainData/dataAllSim.csv"
print("reading data")
xyDataTmp = pd.read_csv(file1)
#print(xyDataTmp.info())
xyData = np.array(xyDataTmp)

x = xyData[:,0:22]
y = xyData[:,22:26]
ylabel = y
y = enc.transform(y).toarray()


del xyDataTmp #节省内存
del xyData #节省内存
#######################3.预测模型
print("3.HMCNF预测模型")
hierarchy = [2,3,5,9]
features_size = x.shape[1]
label_size = y.shape[1]
beta = 0.2

model_name ="hmcnf.h5" 

model = keras.models.load_model(model_name)
y_out = model.predict([x], batch_size=2560)
y_predict = np.where(y_out > 0.5, 1, 0)

predict_ok = np.where(np.sum(y_predict - y, axis=1) == 0, 1, 0)


print("validated {} , {} good out of {} samples".format(model_name, np.sum(predict_ok), predict_ok.shape[0]))
del y_predict #节省内存
del predict_ok #节省内存
#######################3.层次预测预测模型
print("3.层次预测预测模型")
y1 = np.where(y_out[:,0:2] > 0.5, 1, 0)
y2 = np.where(y_out[:,2:5] > 0.5, 1, 0)
y3 = np.where(y_out[:,5:10] > 0.5, 1, 0)
y4 = np.where(y_out[:,10:19] > 0.5, 1, 0)
for i in range(y4.shape[0]):
    tmp1 = y1[i]
    tmp2 = y2[i]
    tmp3 = y3[i]
    tmp4 = y4[i]
    if sum(tmp1) == 0:
        index=  np.argmax(tmp1)
        y1[i,index]=1
        
    if sum(tmp2) == 0:
        index=  np.argmax(tmp2)
        y2[i,index]=1
        
    if sum(tmp3) == 0:
        index=  np.argmax(tmp3)
        y3[i,index]=1
    
    if sum(tmp4) == 0:
        index=  np.argmax(tmp4)
        y4[i,index]=1
        #print(i,y4[i],index)
y_predict = np.concatenate([y1,y2,y3,y4],axis=1)
predict_ok = np.where(np.sum(y_predict - y, axis=1) == 0, 1, 0)
print("validated {} , {} good out of {} samples".format(model_name, np.sum(predict_ok), predict_ok.shape[0]))

#onehot 2 label
ypredict = enc.inverse_transform(y_predict)
del y_predict #节省内存
del predict_ok #节省内存
del y1,y2,y3,y4
#######################4.评估层次模型
#hierarchy = [2,3,5,9]

##第一层，2
print("###################################第一层，2")
h1_yp = ypredict[:,0]
h1_yl = ylabel[:,0]
tmp1 = classification_report(h1_yl,h1_yp)
tmp2 = confusion_matrix(h1_yl,h1_yp,normalize='true')
tmp3 = confusion_matrix(h1_yl,h1_yp,normalize='pred')
print(tmp1)
print(np.around(tmp2, decimals=3))
print(np.around(tmp3, decimals=3))


##第二层，3
print("################################第二层，3")
h2_yp = ypredict[:,1]
h2_yl = ylabel[:,1]
tmp1 = classification_report(h2_yl,h2_yp)
tmp2 = confusion_matrix(h2_yl,h2_yp,normalize='true')
tmp3 = confusion_matrix(h2_yl,h2_yp,normalize='pred')
print(tmp1)
print(np.around(tmp2, decimals=3))
print(np.around(tmp3, decimals=3))



##第三层，5
print("#############################第三层，5")
h3_yp = ypredict[:,2]
h3_yl = ylabel[:,2]
tmp1 = classification_report(h3_yl,h3_yp)
tmp2 = confusion_matrix(h3_yl,h3_yp,normalize='true')
tmp3 = confusion_matrix(h3_yl,h3_yp,normalize='pred')
print(tmp1)
print(np.around(tmp2, decimals=3))
print(np.around(tmp3, decimals=3))


##第四层，9
print("#############################第四层，9")
h4_yp = ypredict[:,3]
h4_yl = ylabel[:,3]
tmp1 = classification_report(h4_yl,h4_yp)
tmp2 = confusion_matrix(h4_yl,h4_yp,normalize='true')
tmp3 = confusion_matrix(h4_yl,h4_yp,normalize='pred')
print(tmp1)
print(np.around(tmp2, decimals=3))
print(np.around(tmp3, decimals=3))



reading data
3.HMCNF预测模型
validated hmcnf.h5 , 810146 good out of 844538 samples
3.层次预测预测模型
validated hmcnf.h5 , 840260 good out of 844538 samples
###################################第一层，2
              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00    552177
         1.0       0.99      0.99      0.99    292361

    accuracy                           0.99    844538
   macro avg       0.99      0.99      0.99    844538
weighted avg       0.99      0.99      0.99    844538

[[0.994 0.006]
 [0.007 0.993]]
[[0.996 0.012]
 [0.004 0.988]]
################################第二层，3
              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00    552177
         1.0       0.97      0.99      0.98    268796
         2.0       0.98      0.76      0.85     23565

    accuracy                           0.99    844538
   macro avg       0.98      0.92      0.94    844538
weighted avg       0.99      0.99      0.99    844538

[[0.9

In [126]:
#第四步，根据混淆矩阵进行聚类。第一列代表识别为类别1的样本真实的类别分布
import numpy as np
import copy
#########################################手动准备模拟数据
mat1 = np.array([[0.952,0.004,0.015,0.008],
 [0.018,0.923,0.016,0.032],
 [0.016,0.036,0.934,0.047],
 [0.014,0.037,0.035,0.913]])
accy = [mat1[0,0],mat1[1,1],mat1[2,2],mat1[3,3]]
print("accuracy",accy)
matT1 = mat1
mat1[:,0] = matT1[:,0]*1000
mat1[:,1] = matT1[:,1]*1000
mat1[:,2] = matT1[:,2]*1000
mat1[:,3] = matT1[:,3]*1000
sumTmp =  sum(mat1)
print(sumTmp)
print(mat1)

##########################################计算最佳合并位置，根据最大的正确率提高
def computeAccuracyDiff(mat1,accy):
    h,w = mat1.shape
    tmp = np.zeros((h-1,w-1))
    matTmp={}
    ##从0到最后，行列合并
    for index in range(h-1):

        tmp = np.zeros((h-1,w))
        num = 0
        ####行合并
        for i in range(h):#行合并
            if i == index:
                tmp[num]=mat1[i]+mat1[i+1]
                num=num+1
                continue
            if i== index+1:
                continue

            tmp[num]=mat1[i]
            num=num+1

        ####列合并   
        mat2=tmp
        tmp = np.zeros((h-1,w-1))
        num = 0
        for j in range(w):#列合并
            if j == index:
                tmp[:,num] = mat2[:,j]+mat2[:,j+1]
                num=num+1
                continue
            if j== index+1:
                continue

            tmp[:,num] = mat2[:,j]
            num=num+1
        matTmp[index] = tmp
        
        #print("合并后的所有矩阵")
        #print(index,matTmp[index])#合并后的所有矩阵
    matTmp1 = copy.deepcopy(matTmp)
   
    ##归一化   
    maxDiffMat = np.zeros((len(matTmp),1))
    for i in range(len(matTmp)):
        tmp = matTmp[i]
        sumTmp =  sum(tmp)
        for j in range(tmp.shape[1]):
            tmp[:,j] = tmp[:,j]/(sumTmp[j])

        accyNow = tmp[i,i]
        maxDiffMat[i]= max(accyNow-accy[i],accyNow-accy[i+1])
    maxIndex = np.argmax(maxDiffMat)
    maxDiff = max(maxDiffMat)
    #print(matTmp1)
    return maxIndex,maxDiff,maxDiffMat,matTmp,matTmp1

#为了思考，不用for循环，直接一步一步做
#4到3
maxIndex,maxDiff,maxDiffMat,matTmp,matTmp1 =computeAccuracyDiff(mat1,accy)
chosedMat =  matTmp1[maxIndex]
print("最佳合并点和矩阵",maxIndex,maxDiff)
print(matTmp1[maxIndex])

#3到2
mat3to2=matTmp1[maxIndex]
accy3to2 = [matTmp[maxIndex][0,0],matTmp[maxIndex][1,1],matTmp[maxIndex][2,2]]
maxIndex3to2,maxDiff3to2,maxDiffMat3to2,matTmp3to2,matTmp3to2Origin =computeAccuracyDiff(mat3to2,accy3to2)
chosedMat = matTmp3to2Origin[maxIndex3to2]
print("最佳合并点和矩阵",maxIndex3to2,maxDiff3to2)
print(matTmp3to2Origin[maxIndex3to2])
print(matTmp3to2[maxIndex3to2])


#########################################采用数据进行分析
print("\n\n\n###################################################")
print("\n\n\n 用第5层数据进行分析")
h3_yp = ypredict[:,2]
h3_yl = ylabel[:,2]
mat1 = confusion_matrix(h3_yl,h3_yp)
p1 = confusion_matrix(h3_yl,h3_yp,normalize='pred')
sumTmp = sum(mat1)
print(mat1)
print(sumTmp)
print(np.around(p1, decimals=3))

########5->4
accy = [p1[0,0],p1[1,1],p1[2,2],p1[3,3],p1[4,4]]
maxIndex,maxDiff,maxDiffMat,matTmp,matTmp1 =computeAccuracyDiff(mat1,accy)
chosedMat =  matTmp1[maxIndex]
print("\n\n\n5->4,最佳合并点和矩阵",maxIndex,maxDiff)
print(matTmp1[maxIndex])
print(np.around(matTmp[maxIndex], decimals=3))

########4->3
mat1=matTmp1[maxIndex]
accy= [matTmp[maxIndex][0,0],matTmp[maxIndex][1,1],matTmp[maxIndex][2,2],matTmp[maxIndex][3,3]]

maxIndex,maxDiff,maxDiffMat,matTmp,matTmp1 =computeAccuracyDiff(mat1,accy)
chosedMat =  matTmp1[maxIndex]
print("\n\n\n4->3,最佳合并点和矩阵",maxIndex,maxDiff)
print(matTmp1[maxIndex])
print(np.around(matTmp[maxIndex], decimals=3))


########3->2
mat1=matTmp1[maxIndex]
accy= [matTmp[maxIndex][0,0],matTmp[maxIndex][1,1],matTmp[maxIndex][2,2]]

maxIndex,maxDiff,maxDiffMat,matTmp,matTmp1 =computeAccuracyDiff(mat1,accy)
chosedMat =  matTmp1[maxIndex]
print("\n\n\n3->2,最佳合并点和矩阵",maxIndex,maxDiff)
print(matTmp1[maxIndex])
print(np.around(matTmp[maxIndex], decimals=3))

accuracy [0.952, 0.923, 0.934, 0.913]
[1000. 1000. 1000. 1000.]
[[952.   4.  15.   8.]
 [ 18. 923.  16.  32.]
 [ 16.  36. 934.  47.]
 [ 14.  37.  35. 913.]]
最佳合并点和矩阵 2 [0.0515]
[[ 952.    4.   23.]
 [  18.  923.   48.]
 [  30.   73. 1929.]]
最佳合并点和矩阵 1 [0.068]
[[ 952.   27.]
 [  48. 2973.]]
[[0.952 0.009]
 [0.048 0.991]]



###################################################



 用第5层数据进行分析
[[550806    893    478      0      0]
 [  3880 157781   6680    231      0]
 [  3716  10625  85590    223     70]
 [  1221    683    900  13184    214]
 [   300    168    393     60   6442]]
[559923 170150  94041  13698   6726]
[[0.984 0.005 0.005 0.    0.   ]
 [0.007 0.927 0.071 0.017 0.   ]
 [0.007 0.062 0.91  0.016 0.01 ]
 [0.002 0.004 0.01  0.962 0.032]
 [0.001 0.001 0.004 0.004 0.958]]



5->4,最佳合并点和矩阵 1 [0.07656029]
[[5.50806e+05 1.37100e+03 0.00000e+00 0.00000e+00]
 [7.59600e+03 2.60676e+05 4.54000e+02 7.00000e+01]
 [1.22100e+03 1.58300e+03 1.31840e+04 2.14000e+02]
 [3.00000e+02 5.61000e+02 6.0

In [112]:
h3_yp = ypredict[:,2]
h3_yl = ylabel[:,2]
o1 = confusion_matrix(h3_yl,h3_yp)
p1 = confusion_matrix(h3_yl,h3_yp,normalize='pred')
print(o1)
print(sum(o1))
print(np.around(p1, decimals=3))


[[550806    893    478      0      0]
 [  3880 157781   6680    231      0]
 [  3716  10625  85590    223     70]
 [  1221    683    900  13184    214]
 [   300    168    393     60   6442]]
[559923 170150  94041  13698   6726]
[[0.984 0.005 0.005 0.    0.   ]
 [0.007 0.927 0.071 0.017 0.   ]
 [0.007 0.062 0.91  0.016 0.01 ]
 [0.002 0.004 0.01  0.962 0.032]
 [0.001 0.001 0.004 0.004 0.958]]


NameError: name 'cls' is not defined